In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ankurbhatt1611/WineQualityDataScienceProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ankurbhatt1611"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "896696bb520653b1b87471269f6c9d55771f4bcc"


In [15]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvaluationConfig:
    root_dir : Path 
    test_data_path : Path
    model_path : Path 
    metric_file_name : Path 
    all_params : dict 
    target_column : Path 
    mlflow_uri : str 




In [ ]:
# %pwd
# os.chdir("../")
# %pwd

'd:\\WineQuality_EndToEndDataScienceProject\\WineQualityDataScienceProject'

In [13]:
from src.WineQualityPrediction.constants import * 
from src.WineQualityPrediction.utils.common import read_yaml , create_directories

In [18]:

class ConfigurationManager:
    def __init__(self , config_file_path = config_file_path,
                  params_file_path  = params_file_path,
                    schema_file_path = schema_file_path):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig :

        create_directories([self.config.model_evaluation.root_dir])
        config = self.config.model_evaluation
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = self.params.ElasticNet,
            target_column = self.schema.TARGET_COLUMN.name,
            mlflow_uri = "https://dagshub.com/ankurbhatt1611/WineQualityDataScienceProject.mlflow"
            
        )
        return model_evaluation_config




In [21]:
import pandas as pd
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score 
from urllib.parse import urlparse
import mlflow 
import mlflow.sklearn 
import numpy as np 
import joblib 
import os 
from src.WineQualityPrediction.utils.common import save_json

class ModelEvaluation:

    def __init__(self , config : ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self , actual , pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual , pred)
        return rmse , mae , r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column] , axis = 1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse , mae , r2 ) = self.eval_metrics(test_y , predicted_qualities)

            # saving metrics in local
            scores = {'rmse':rmse , 'mae' : mae , 'r2' : r2}
            save_json(path = Path(self.config.metric_file_name) , data = scores)

            # log params and metrics in mlflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            # Register the model
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model,'model',registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [22]:
try :
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e




[2025-04-17 22:35:51,394] : INFO : common : yaml file: config\config.yml loaded successfully
[2025-04-17 22:35:51,397] : INFO : common : yaml file: params.yml loaded successfully
[2025-04-17 22:35:51,400] : INFO : common : yaml file: schema.yml loaded successfully
[2025-04-17 22:35:51,401] : INFO : common : Created directory at: artifacts
[2025-04-17 22:35:51,401] : INFO : common : Created directory at: artifacts/model_evaluation
[2025-04-17 22:35:52,244] : INFO : common : Json file saved at: artifacts\model_evaluation\metrics.json


2025/04/17 22:36:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/04/17 22:36:06 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run languid-shrimp-517 at: https://dagshub.com/ankurbhatt1611/WineQualityDataScienceProject.mlflow/#/experiments/0/runs/b56ab30cc5fa411b820111b31878ebad
🧪 View experiment at: https://dagshub.com/ankurbhatt1611/WineQualityDataScienceProject.mlflow/#/experiments/0
